In [114]:
import pandas as pd
import numpy as np

Cargo los dataset

In [115]:
coursera_info = pd.read_csv("Data\Coursera_courses.csv")
coursera_reviews = pd.read_csv("Data\Coursera_reviews.csv")
edx_info = pd.read_csv("Data\edx_courses.csv")
udemy_info = pd.read_csv("Data/udemy_courses.csv")

In [116]:
coursera_info.shape

(623, 4)

In [117]:
edx_info.shape

(975, 16)

In [118]:
udemy_info.shape

(3678, 12)

Reviso todas las columnas de cada dataset 

In [119]:
print(f"columnas coursera {coursera_info.columns}\n"
      f"columnas coursera reviews{coursera_reviews.columns}\n"
      f"columnas edx {edx_info.columns}\n"
      f"columnas udemy {udemy_info.columns}")

columnas coursera Index(['name', 'institution', 'course_url', 'course_id'], dtype='object')
columnas coursera reviewsIndex(['reviews', 'reviewers', 'date_reviews', 'rating', 'course_id'], dtype='object')
columnas edx Index(['title', 'summary', 'n_enrolled', 'course_type', 'institution',
       'instructors', 'Level', 'subject', 'language', 'subtitles',
       'course_effort', 'course_length', 'price', 'course_description',
       'course_syllabus', 'course_url'],
      dtype='object')
columnas udemy Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject'],
      dtype='object')


## Cargo los dataset extra para completar la información

In [120]:
coursera_completo = pd.read_csv("Data\webautomation_coursera.csv")
coursera_price_level = pd.read_csv("Data\coursera_price_level.csv")
coursera_student = pd.read_csv("Data\Coursera_students.csv")
udemy_extra = pd.read_csv("Data/udemy_extra.csv")

## Genero las transformaciones de las diferentes columnas

### Coursera

In [121]:
coursera_info.columns

Index(['name', 'institution', 'course_url', 'course_id'], dtype='object')

In [122]:
# Me quedo con las columnas que voy a usar
coursera = coursera_info[["course_id", "name"]]

In [123]:
coursera.drop_duplicates(inplace=True)
coursera_reviews.drop_duplicates(inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\2059813281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coursera.drop_duplicates(inplace=True)


In [124]:
#Columna nueva generando un promedio de los rating del dataset reviews
ratings_coursera = round(coursera_reviews.groupby("course_id")['rating'].mean(),1)

In [125]:
# Hago un merge con los promedios de ratings que cree
coursera = pd.merge(coursera, ratings_coursera, on= "course_id", how= "left")

In [126]:
ratings_coursera

course_id
aboriginal-education                                        4.6
access-control-sscp                                         4.5
accounting-analytics                                        4.4
accounting-data-analytics-python                            3.5
actualizacion-manejo-diabetes-tipo-2                        4.8
                                                           ... 
womens-health-human-rights                                  4.9
write-a-feature-length-screenplay-for-film-or-television    4.6
write-your-first-novel                                      3.8
writing-editing-words                                       4.7
writing-for-business                                        4.8
Name: rating, Length: 604, dtype: float64

Generar un nuevo dataset con los datos de alumnos que reseñaron los cursos por año asumiendo que lo reseñan cuando terminan

In [127]:
# Convertir la columna 'Fecha' al tipo de datos de fecha
coursera_reviews['Fecha'] = pd.to_datetime(coursera_reviews['date_reviews'])

In [128]:
# Extraer el mes y el año en columnas separadas
coursera_reviews['Mes'] = coursera_reviews['Fecha'].dt.strftime('%B')
coursera_reviews['Año'] = coursera_reviews['Fecha'].dt.strftime('%Y')

In [129]:
#Agrupar por curso y contar la cantidad por año
alumnos_por_anio = coursera_reviews.groupby(["course_id","Año"]).size().reset_index(name='Cantidad')

In [130]:
# Hago un merge con coursera para que me queden los nombres de los cursos y no los id
alumnos_por_anio= pd.merge(alumnos_por_anio, coursera[["course_id", "name"]], on= "course_id", how= "left")

In [131]:
alumnos_por_anio.rename(columns={"name": "course_title", }, inplace=True)

In [132]:
#Exportar el archivo
alumnos_por_anio.to_csv("alumnos_por_año.csv", index= False)

Busco que columnas me interesan de los data set extra

In [133]:
coursera_completo.columns

Index(['url', 'title', 'associated-university-institution-company', 'type',
       'image', 'category-subject-area', 'certificate-is-available',
       'description', 'duration', 'language', 'level', 'prerequisites',
       'price', 'rating', 'syllabus', 'timestamp'],
      dtype='object')

In [134]:
coursera_price_level.columns

Index(['url', 'price', 'course_by', 'title', 'skills', 'ratings', 'reviews',
       'level, type and duration'],
      dtype='object')

In [135]:
coursera_student.columns

Index(['Unnamed: 0', 'course_title', 'course_organization',
       'course_Certificate_type', 'course_rating', 'course_difficulty',
       'course_students_enrolled'],
      dtype='object')

Uno los data set

In [136]:
# course_students_enrolled
coursera = pd.merge(coursera, coursera_student[["course_title", "course_students_enrolled"]], left_on= "name", right_on="course_title", how= "left")

In [137]:
# Level, lenguage, levele,type and duration
coursera = pd.merge(coursera, coursera_completo[["title","level",'language', "category-subject-area"]], left_on="name", right_on="title", how= "left")
coursera = pd.merge(coursera, coursera_price_level[["title","level, type and duration",]], left_on="name", right_on="title", how= "left")

In [138]:
coursera

,course_id,name,rating,course_title,course_students_enrolled,title_x,level,language,category-subject-area,title_y,"level, type and duration"
0,machine-learning,Machine Learning,4.7,Machine Learning,290k,NaN,NaN,NaN,NaN,NaN,NaN
1,machine-learning,Machine Learning,4.7,Machine Learning,3.2m,NaN,NaN,NaN,NaN,NaN,NaN
2,indigenous-canada,Indigenous Canada,4.8,Indigenous Canada,51k,NaN,NaN,NaN,NaN,Indigenous Canada,Mixed · Course · 1-3 Months
3,the-science-of-well-being,The Science of Well-Being,4.9,The Science of Well-Being,2.5m,NaN,NaN,NaN,NaN,The Science of Well-Being,Mixed · Course · 1-3 Months
4,technical-support-fundamentals,Technical Support Fundamentals,4.8,Technical Support Fundamentals,280k,Technical Support Fundamentals,Beginner Level,English,Support and Operations,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
639,accounting-data-analytics-python,Accounting Data Analytics with Python,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,spectroscopy,Introduction to Molecular Spectroscopy,4.6,NaN,NaN,NaN,NaN,NaN,NaN,Introduction to Molecular Spectroscopy,Intermediate · Course · 1-4 Weeks
641,managing-as-a-coach,Managing as a Coach,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,hotel-distribution,The fundamentals of hotel distribution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Realizo las transformaciones necesarias

In [139]:
# Me quedo solo con la primer palabra de la columna
coursera["level, type and duration"] = coursera['level, type and duration'].str.extract('(\w+)')

In [140]:
# Utilizar fillna() para reemplazar los valores nulos en 'level'
coursera["level"] = coursera["level"].fillna(coursera["level, type and duration"])

In [141]:
#Normalizo los valores de level como los de udemy
coursera.loc[:, "level"].replace({"Mixed" : "All", "Beginner Level": "Beginner", "Intermediate Level" : "Intermediate", "-": np.nan, "Advanced": "Expert"}, inplace= True)

In [142]:
#Normalizo los valores de la columna course_students_enrolled	
coursera['course_students_enrolled'] = coursera['course_students_enrolled'].str.replace('.', '').str.replace('k', '000').str.replace('m', '000000')

# Convertir la columna a tipo numérico
coursera['course_students_enrolled']= pd.to_numeric(coursera['course_students_enrolled'])

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\2394513039.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  coursera['course_students_enrolled'] = coursera['course_students_enrolled'].str.replace('.', '').str.replace('k', '000').str.replace('m', '000000')


In [143]:
coursera["language"].unique()

array([nan, 'English', 'Spanish', 'Portuguese (Brazilian)'], dtype=object)

In [144]:
#Normalizo los valores language 
coursera.loc[:, "language"].replace({'Portuguese (Brazilian)':'Portuguese'}, inplace= True)

In [145]:
coursera

,course_id,name,rating,course_title,course_students_enrolled,title_x,level,language,category-subject-area,title_y,"level, type and duration"
0,machine-learning,Machine Learning,4.7,Machine Learning,290000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,machine-learning,Machine Learning,4.7,Machine Learning,32000000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,indigenous-canada,Indigenous Canada,4.8,Indigenous Canada,51000.0,NaN,All,NaN,NaN,Indigenous Canada,Mixed
3,the-science-of-well-being,The Science of Well-Being,4.9,The Science of Well-Being,25000000.0,NaN,All,NaN,NaN,The Science of Well-Being,Mixed
4,technical-support-fundamentals,Technical Support Fundamentals,4.8,Technical Support Fundamentals,280000.0,Technical Support Fundamentals,Beginner,English,Support and Operations,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
639,accounting-data-analytics-python,Accounting Data Analytics with Python,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,spectroscopy,Introduction to Molecular Spectroscopy,4.6,NaN,NaN,NaN,Intermediate,NaN,NaN,Introduction to Molecular Spectroscopy,Intermediate
641,managing-as-a-coach,Managing as a Coach,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,hotel-distribution,The fundamentals of hotel distribution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
# Elimino la columna title que viene del data set extra
coursera.drop(["title_x", "course_title", "title_y", "level, type and duration"], axis=1, inplace= True)
coursera.rename(columns={"course_id": "Id", "course_students_enrolled": "num_subscribers", "name": "course_title", "category-subject-area": "category"}, inplace=True)

Agrego columna con el nombre de la plataforma para referencia

In [147]:
coursera["plataforma"] = "Coursera"

In [148]:
coursera.isnull().sum()

Id                   0
course_title         0
rating              20
num_subscribers    153
level              390
language           590
category           590
plataforma           0
dtype: int64

In [149]:
coursera.shape

(644, 8)

Elimino los que no tengan numero de suscriptores ya que como uní los datos con otro dataset, que no tenga número de suscriptores podría significar que el curso no aparecía en el data set extra y no que el curso no tiene alumnos

In [150]:
coursera.dropna(subset="num_subscribers", inplace= True)

### Udemy

In [151]:
udemy_info.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject'],
      dtype='object')

In [152]:
# Me quedo con las columnas que voy a utilizar
udemy = udemy_info[['course_id', 'course_title','is_paid', 'price',
       'num_subscribers', 'level', "content_duration", "published_timestamp"]]

In [153]:
#Elimino duplicados
udemy.drop_duplicates(inplace=True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\920758881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  udemy.drop_duplicates(inplace=True)


Busco las columnas que me interesan del data set extra

In [154]:
udemy_extra.columns

Index(['id', 'title', 'is_paid', 'price', 'headline', 'num_subscribers',
       'avg_rating', 'num_reviews', 'num_comments', 'num_lectures',
       'content_length_min', 'published_time', 'last_update_date', 'category',
       'subcategory', 'topic', 'language', 'course_url', 'instructor_name',
       'instructor_url'],
      dtype='object')

In [155]:
#Agrego las columnas del data set extra
udemy = pd.merge(udemy, udemy_extra[['id','avg_rating', 'language', 'category']], left_on="course_id", right_on="id", suffixes=('', '_extra'))


Realizo las transformaciones de formato necesarias

In [156]:
#Elimino el id que viene del merge y le cambio el nombre a la columna course_id
udemy.drop(["id"], axis= 1, inplace= True)
udemy.rename(columns={"course_id": "Id", 'avg_rating': "rating"}, inplace=True)

In [157]:
#Miro los valores únicos de la columna 
udemy["level"].unique()

array(['All Levels', 'Intermediate Level', 'Beginner Level',
       'Expert Level'], dtype=object)

In [158]:
# Elimino las palabras Level y Levels, luego elimino posibles espacios en blanco
udemy["level"] = udemy["level"].str.replace("Levels",'').str.replace("Level",'').str.strip()

In [159]:
# Redondeo rating a un decimal
udemy["rating"] = round(udemy["rating"],1)

In [160]:
udemy["language"].unique()

array(['English', 'Spanish', 'German', 'Arabic', 'Japanese', 'French',
       'Portuguese', 'Russian', 'Simplified Chinese', 'Hindi', 'Italian',
       'Korean', 'Turkish', 'Urdu'], dtype=object)

In [161]:
#Normalizo los valores language 
udemy.loc[:, "language"].replace({'Simplified Chinese':'Chinese'}, inplace= True)

In [162]:
udemy["published_timestamp"][0]


'2017-01-18T20:58:58Z'

In [163]:
#Genero una columna nueva a partir de la fecha de publicación del curso dejando solo el año 
udemy['published_timestamp'] = pd.to_datetime(udemy['published_timestamp'])
udemy["anio_publicacion"] = udemy['published_timestamp'].dt.year
udemy.drop(["published_timestamp"], axis=1, inplace=True)

In [164]:
#Modifico los valores de is_paid por ceros y unos
udemy.loc[:, "is_paid"].replace({True: "Si", False: "No"}, inplace= True)

Agrego columna de ingresos por curso

In [165]:
udemy["ingreso_por_curso"] = udemy["price"] * udemy["num_subscribers"]

Agrego columna para tener referencia de que plataforma hace referencia

In [166]:
udemy["plataforma"] = "Udemy"

In [167]:
# Miro los datos nulos
udemy.isnull().sum()

Id                   0
course_title         0
is_paid              0
price                0
num_subscribers      0
level                0
content_duration     0
rating               0
language             0
category             0
anio_publicacion     0
ingreso_por_curso    0
plataforma           0
dtype: int64

### edX

In [168]:
edx_info.columns

Index(['title', 'summary', 'n_enrolled', 'course_type', 'institution',
       'instructors', 'Level', 'subject', 'language', 'subtitles',
       'course_effort', 'course_length', 'price', 'course_description',
       'course_syllabus', 'course_url'],
      dtype='object')

In [169]:
edx = edx_info[['title','n_enrolled','Level','subject', 'language','price', "course_length"]]

In [170]:
#Elimino duplicados
edx.drop_duplicates(inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\10579408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.drop_duplicates(inplace= True)


In [171]:
edx.rename(columns={'title': 'course_title', "n_enrolled":"num_subscribers", 'Level': 'level', 'subject': "category", "course_length": "content_duration"}, inplace=True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\2335420912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.rename(columns={'title': 'course_title', "n_enrolled":"num_subscribers", 'Level': 'level', 'subject': "category", "course_length": "content_duration"}, inplace=True)


Realizo las transformaciones necesarias

In [172]:
# Reemplazo la coma por nada en num_subscribers
edx["num_subscribers"] = pd.to_numeric(edx["num_subscribers"].str.replace(",", ""), errors='coerce').astype(pd.Int64Dtype())

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\3817391365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx["num_subscribers"] = pd.to_numeric(edx["num_subscribers"].str.replace(",", ""), errors='coerce').astype(pd.Int64Dtype())


In [173]:
# Extraigo de la columna price, por un lado si es gratuito, por otro si tiene posibilidad de certificación y cuanto vale 
edx.loc[:,'add_certificate'] = edx['price'].str.contains('FREE', case=False)
edx.loc[:,'price_Certificate'] = edx['price'].str.extract(r'\$(\d+)').astype(int)
edx['is_paid'] = np.where(edx['add_certificate'], False, True)


C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\3672510344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:,'add_certificate'] = edx['price'].str.contains('FREE', case=False)
C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\3672510344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:,'price_Certificate'] = edx['price'].str.extract(r'\$(\d+)').astype(int)
C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\3672510344.py:4: SettingWithCopyWarning: 
A value is trying to be set o

In [174]:
#Modifico los valores de is_paid por si y no
edx.loc[:, "is_paid"].replace({True: "Si", False: "No"}, inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\2797811582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:, "is_paid"].replace({True: "Si", False: "No"}, inplace= True)


In [175]:
#Modifico los valores de add_certificate por si y no
edx.loc[:, "add_certificate"].replace({True: "Si", False: "No"}, inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\697017449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:, "add_certificate"].replace({True: "Si", False: "No"}, inplace= True)


In [176]:
edx

,course_title,num_subscribers,level,category,language,price,content_duration,add_certificate,price_Certificate,is_paid
0,How to Learn Online,124980,Introductory,Education & Teacher Training,English,FREE-Add a Verified Certificate for $49 USD,2 Weeks,Si,49,No
1,Programming for Everybody (Getting Started wit...,293864,Introductory,Computer Science,English,FREE-Add a Verified Certificate for $49 USD,7 Weeks,Si,49,No
2,CS50's Introduction to Computer Science,2442271,Introductory,Computer Science,English,FREE-Add a Verified Certificate for $90 USD,12 Weeks,Si,90,No
3,The Analytics Edge,129555,Intermediate,Data Analysis & Statistics,English,FREE-Add a Verified Certificate for $199 USD,13 Weeks,Si,199,No
4,Marketing Analytics: Marketing Measurement Str...,81140,Introductory,Computer Science,English,FREE-Add a Verified Certificate for $249 USD,4 Weeks,Si,249,No
...,...,...,...,...,...,...,...,...,...,...
970,Leaders in Citizen Security and Justice Manage...,<NA>,Intermediate,Social Sciences,English,FREE-Add a Verified Certificate for $25 USD,10 Weeks,Si,25,No
971,Pattern Studying and Making | 图案审美与创作,<NA>,Introductory,Art & Culture,中文,FREE-Add a Verified Certificate for $139 USD,12 Weeks,Si,139,No
972,Computational Neuroscience: Neuronal Dynamics ...,11246,Advanced,Biology & Life Sciences,English,FREE-Add a Verified Certificate for $139 USD,6 Weeks,Si,139,No
973,Cities and the Challenge of Sustainable Develo...,8775,Introductory,Environmental Studies,English,FREE-Add a Verified Certificate for $25 USD,1 Weeks,Si,25,No


In [177]:
# Cambio los valores del precio ya que si es Gratis deberá ser nulo
edx.loc[edx["is_paid"] == "No", "price"] = 0

In [178]:
#Normalizo los valores de level como los de udemy
edx.loc[:, "level"].replace({"Introductory": "Beginner", "Advanced" : "Expert"}, inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\778010007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:, "level"].replace({"Introductory": "Beginner", "Advanced" : "Expert"}, inplace= True)


In [179]:
#Normalizo la duración a horas
edx["content_duration"] = edx["content_duration"].str.replace(' Weeks', '').astype(int) * 7 * 24


C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\287060545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx["content_duration"] = edx["content_duration"].str.replace(' Weeks', '').astype(int) * 7 * 24


In [180]:
edx["language"].unique()

array(['English', 'Español', 'Italiano', '日本語', 'Français', '中文',
       'Português', 'اللغة العربية', 'Deutsch'], dtype=object)

In [181]:
#Normalizo los valores language 
edx.loc[:, "language"].replace({'Español': 'Spanish','Italiano' : 'Italian', '日本語':'Japanese', 'Français': 'French', '中文':'Chinese','Português':'Portuguese', 'Deutsch': 'German'}, inplace= True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\1015654758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.loc[:, "language"].replace({'Español': 'Spanish','Italiano' : 'Italian', '日本語':'Japanese', 'Français': 'French', '中文':'Chinese','Português':'Portuguese', 'Deutsch': 'German'}, inplace= True)


Agrego columna para tener referencia de que plataforma hace referencia

In [182]:
edx["plataforma"] = "edX"

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\2085601199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx["plataforma"] = "edX"


In [183]:
edx.isnull().sum()

course_title           0
num_subscribers      120
level                  0
category               0
language               0
price                  0
content_duration       0
add_certificate        0
price_Certificate      0
is_paid                0
plataforma             0
dtype: int64

Elimino los que no tengan numero de suscriptores porque el enfoque que le voy a dar a este análisis proviene de la cantidad de alumnos por curso

In [184]:
edx.dropna(subset="num_subscribers", inplace=True)

C:\Users\Clarita\AppData\Local\Temp\ipykernel_10952\3494020482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edx.dropna(subset="num_subscribers", inplace=True)


## Concateno los 3 dataset para empezar con el EDA

In [185]:
data = pd.concat([udemy, edx, coursera], join= "outer", ignore_index= True)

In [186]:
data['Id'] = pd.RangeIndex(start=1, stop=len(data) + 1)

In [187]:
data = data.drop(data[data['Id'].isin([1262, 1363, 1146, 1348])].index)

In [188]:
data.to_csv("cursos.csv", index= False)

In [189]:
udemy.to_csv("udemy.csv", index= False)

In [190]:
edx.to_csv("edx.csv", index= False)

In [191]:
coursera.to_csv("coursera.csv", index= False)